In [ ]:
import chess
board = chess.Board()
board.push_san("e4")
board.push_san("e5")
board.push_san("Qh5")
board.push_san("Nc6")
# print([*map(str, board.legal_moves)])
board.push_san("Bc4")
board.push_san("Nf6")
board.push_san("Qxf7")
print(board.outcome().winner)
print(board.fen())
board.mirror()

## Generate all possible legal moves

In [ ]:
action_space = set()
board = chess.Board()
for i in chess.SQUARES:
    for color in [chess.WHITE, chess.BLACK]:
        for piece in [chess.PAWN, chess.QUEEN, chess.KNIGHT, chess.KING]:
            board.clear()
            board.turn = color
            board.set_piece_at(i, chess.Piece(piece, color))
            action_space = action_space.union({*map(str, board.pseudo_legal_moves)})
board.clear()
for i in range(0, 8):
    board.set_piece_at(i, chess.Piece(chess.ROOK, chess.WHITE))
for i in range(56, 64):
    board.set_piece_at(i, chess.Piece(chess.ROOK, chess.BLACK))
for i in range(8, 16):
    board.set_piece_at(i, chess.Piece(chess.PAWN, chess.BLACK))
for i in range(48, 56):
    board.set_piece_at(i, chess.Piece(chess.PAWN, chess.WHITE))
board.turn=True
action_space = action_space.union({*map(str, board.pseudo_legal_moves)})
board.turn = False
action_space = action_space.union({*map(str, board.pseudo_legal_moves)})
with open('chess_possible_moves.txt', 'w') as f:
    f.write('\n'.join(sorted(action_space)))
print(len(action_space))

## Board Representation

In [ ]:
def mirror_uci(move):
    file_map = {'a': 'h', 'b': 'g', 'c': 'f', 'd': 'e', 
                'e': 'd', 'f': 'c', 'g': 'b', 'h': 'a'}
    rank_map = {'1': '8', '2': '7', '3': '6', '4': '5', 
                '5': '4', '6': '3', '7': '2', '8': '1'}

    mirrored_move = ""
    
    for char in move:
        if char in file_map:
            mirrored_move += file_map[char]
        elif char in rank_map:
            mirrored_move += rank_map[char]
        else:
            mirrored_move += char 
    
    return mirrored_move

print(mirror_uci("e2e4"))
print(mirror_uci("a7a8q")) 


In [ ]:
import numpy as np
state = chess.Board()
state.push_uci("e2e4")
matrices = []

if not state.turn:
    board = state.mirror()

symbols = ['P', 'R', 'K', 'B', 'Q', 'K', 'p', 'r', 'k', 'b', 'q', 'k']

for symbol in symbols:
    matrix = np.zeros((8, 8), dtype=int)
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece and piece.symbol() == symbol:
            i, j = np.unravel_index(square, (8, 8))
            matrix[i][j] = 1
    matrices.append(matrix)

ones = np.ones((8, 8), dtype=int)
zeros = np.zeros((8, 8), dtype=int)
matrices.append(ones if board.has_kingside_castling_rights(chess.WHITE) else zeros)
matrices.append(ones if board.has_queenside_castling_rights(chess.WHITE) else zeros)
matrices.append(ones if board.has_kingside_castling_rights(chess.BLACK) else zeros)
matrices.append(ones if board.has_queenside_castling_rights(chess.BLACK) else zeros)

en_passant = np.zeros((8, 8), dtype=int)
if board.ep_square is not None:
    i, j = np.unravel_index(board.ep_square, (8, 8))
    en_passant[i][j] = 1
matrices.append(en_passant)

halfmove_clock = np.full((8, 8), board.halfmove_clock, dtype=int)
matrices.append(halfmove_clock)

matrices = np.stack(matrices, axis=0)
print(len(matrices))